<a href="https://colab.research.google.com/github/gaesu96/gaesu_public/blob/main/%08blur_warship_faster_rcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch

# 작업환경이 mmdetection 2.x 기준으로, pytorch 1.13.0으로 setting(mmdetection 2.x는 pytorch 2.0대를 지원하지 않음)
!pip install torch==1.13.0+cu116 torchvision==0.14.0+cu116 --extra-index-url https://download.pytorch.org/whl/cu116
# mmcv를 위해서 mmcv-full을 먼저 설치
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu116/torch1.13/index.html
# mmdetection은 3.0 version이 출시되었으나, 실습 환경 상 2.x대로 설치
!git clone --branch 2.x https://github.com/open-mmlab/mmdetection.git
!cd mmdetection; python setup.py install

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 787.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 26.5 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.2.1+cu121
    Uninstalling torch-2.2.1+cu121:
      Successfully uninstalled torch-2.2.1+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.17.1+cu121
    Uninstalling torchvision-0.17.1+cu121:
      Successfully uninstalled torchvision-0.17.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 1.13.0+cu116 which is incompatible.
torchdata 0.7.1 requires torch>=2, but you have torch 1.13.0+cu116 which is incompatible.
torchtext 0.17.1 requires torc

Looking in links: https://download.openmmlab.com/mmcv/dist/cu116/torch1.13/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 7.1 MB/s eta 0:00:00
Cloning into 'mmdetection'...
remote: Enumerating objects: 38019, done.
remote: Total 38019 (delta 0), reused 0 (delta 0), pack-reused 38019
Receiving objects: 100% (38019/38019), 63.25 MiB | 9.87 MiB/s, done.
Resolving deltas: 100% (26222/26222), done.
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        **

In [11]:
!mkdir /content/drive/MyDrive/warship_detect_project/blur_datasets
!unzip -q '/content/drive/MyDrive/warship_detect_project/blur_datasets.zip' -d '/content/drive/MyDrive/warship_detect_project/blur_datasets/'

mkdir: cannot create directory ‘/content/drive/MyDrive/warship_detect_project/blur_datasets’: File exists
replace /content/drive/MyDrive/warship_detect_project/blur_datasets/data.yaml? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
train_img_list = os.listdir('/content/drive/MyDrive/warship_detect_project/blur_datasets/train/images')
val_img_list = os.listdir('/content/drive/MyDrive/warship_detect_project/blur_datasets/valid/images')

for i in train_img_list:
  if i.endswith('.png'):
    old_filepath = osp.join(train_dir, filename)
    new_filepath = osp.join(train_dir, osp.splitext(filename)[0] + '.jpg')
    os.rename(old_filepath, new_filepath)

In [13]:
import os
import os.path as osp

train_dir = '/content/drive/MyDrive/warship_detect_project/blur_datasets/image/train'
val_dir = '/content/drive/MyDrive/warship_detect_project/blur_datasets/image/val'

# Train 이미지 파일의 확장자를 jpg로 변경
for filename in os.listdir(train_dir):
    if filename.endswith('.png'):
        old_filepath = osp.join(train_dir, filename)
        new_filepath = osp.join(train_dir, osp.splitext(filename)[0] + '.jpg')
        os.rename(old_filepath, new_filepath)

# Validation 이미지 파일의 확장자를 jpg로 변경
for filename in os.listdir(val_dir):
    if filename.endswith('.png'):
        old_filepath = osp.join(val_dir, filename)
        new_filepath = osp.join(val_dir, osp.splitext(filename)[0] + '.jpg')
        os.rename(old_filepath, new_filepath)

In [1]:
import mmcv

/usr/local/lib/python3.10/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:
import os

train_img_list = os.listdir('/content/drive/MyDrive/warship_detect_project/blur_datasets/train/images')
val_img_list = os.listdir('/content/drive/MyDrive/warship_detect_project/blur_datasets/valid/images')
train_output_file = '/content/drive/MyDrive/warship_detect_project/blur_datasets/train.txt'
val_output_file = '/content/drive/MyDrive/warship_detect_project/blur_datasets/val.txt'
with open(train_output_file, 'w') as f:
    for i in train_img_list:
        file_name = osp.splitext(i)[0]
        f.write(file_name + '\n')

with open(val_output_file, 'w') as f:
    for i in val_img_list:
        file_name = osp.splitext(i)[0]
        f.write(file_name + '\n')


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/warship_detect_project/blur_datasets/train/images'

In [16]:
lines = mmcv.list_from_file('/content/drive/MyDrive/warship_detect_project/blur_datasets/label/val/bs_32_35_aug_weather_out.txt')


content = [line.strip().split('  ') for line in lines]

bbox_idxes = [x[0] for x in content]
print(content)
#bboxes = [[float(info) for info in x[1:]] for x in content]


[['9', '0.528155', '0.508753', '0.897087', '0.571116'], ['10', '0.217476', '0.7286649999999999', '0.048544000000000004', '0.06564599999999998']]


In [2]:
import copy
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset
import mmcv
import cv2
import numpy as np
import os.path as osp
import torch
@DATASETS.register_module(force=True)
class WarshipDataset(CustomDataset):
  CLASSES = ('DDG', 'DDH', 'FFG', 'FF', 'PCC', 'PKG', 'LST', 'MLS', 'AOE', 'ATS', 'HULL_NUM')

  def load_annotations(self, ann_file):
    print('self.data_root', self.data_root, 'self.ann_file:', ann_file, 'self.img_prefix:')
    cat2label = {k:int(i) for i, k in enumerate(self.CLASSES)}
    image_list = mmcv.list_from_file(self.ann_file)

    data_infos = []

    for image_id in image_list:
      filename = '{0:}/{1:}.jpg'.format(self.img_prefix, image_id)
      image = cv2.imread(filename)
      height, width = image.shape[:2]

      data_info = {'filename': str(image_id) + '.jpg',
                   'width': width, 'height': height}

      label_prefix = self.img_prefix.replace('image', 'label')
      lines = mmcv.list_from_file(osp.join(label_prefix, str(image_id)+'.txt'))
      if 'aug_weather' in image_id:
        content = [line.strip().split('  ') for line in lines]
      else:
        content = [line.strip().split(' ') for line in lines]
      bbox_idxes = [x[0] for x in content]
      bboxes = [[float(info) for info in x[1:]] for x in content]

      gt_bboxes = []
      gt_labels = []
      gt_bboxes_ignore = []
      gt_labels_ignore = []

      for bbox_idx, bbox in zip(bbox_idxes, bboxes):
        left = (bbox[0] - (bbox[2] / 2)) * width
        top = (bbox[1] - (bbox[3] / 2)) * height
        right = (bbox[0] + (bbox[2] / 2)) * width
        bottom = (bbox[1] + (bbox[3] / 2)) * height

        bbox[0] = left
        bbox[1] = top
        bbox[2] = right
        bbox[3] = bottom
        if int(bbox_idx) in cat2label.values():
          gt_bboxes.append(bbox)
          gt_labels.append(bbox_idx)

        else:
          gt_bboxes_ignore.append(bbox)
          gt_labels_ignore.append(-1)

      data_anno = {
          'bboxes': np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
          'labels': np.array(gt_labels, dtype=np.int64),
          'bboxes_ignore': np.array(gt_bboxes_ignore, dtype=np.float32).reshape(-1, 4),
          'labels_ignore': np.array(gt_labels_ignore, dtype=np.int64)
      }

      data_info.update(ann=data_anno)
      data_infos.append(data_info)
    return data_infos


In [3]:
config_file = '/content/mmdetection/configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py'
checkpoint_file = '/content/drive/MyDrive/warship_detect_project/tutorial_exps_normal/latest.pth'

In [4]:
from mmcv import Config

cfg = Config.fromfile(config_file)
print(cfg.pretty_text)

model = dict(
    type='FasterRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.

In [5]:
from mmdet.apis import set_random_seed

cfg.dataset_type = 'WarshipDataset'
cfg.data_root = '/content/drive/MyDrive/warship_detect_project/blur_datasets/'

cfg.data.train.type = 'WarshipDataset'
cfg.data.train.data_root = '/content/drive/MyDrive/warship_detect_project/blur_datasets/'
cfg.data.train.ann_file = 'train.txt'
cfg.data.train.img_prefix = 'image/train'

cfg.data.val.type = 'WarshipDataset'
cfg.data.val.data_root = '/content/drive/MyDrive/warship_detect_project/blur_datasets/'
cfg.data.val.ann_file = 'val.txt'
cfg.data.val.img_prefix = 'image/val'

cfg.data.test.type = 'WarshipDataset'
cfg.data.test.data_root = '/content/drive/MyDrive/warship_detect_project/blur_datasets/'
cfg.data.test.ann_file = 'val.txt'
cfg.data.test.img_prefix = 'image/val'

cfg.model.roi_head.bbox_head.num_classes = 11

cfg.load_from = '/content/drive/MyDrive/warship_detect_project/tutorial_exps_normal/latest.pth'

cfg.work_dir = '/content/drive/MyDrive/warship_detect_project/tutorial_exps_normal/blur_datasets/'
cfg.optimizer = dict(type='SGD', lr=0.001 / 8, momentum = 0.9, weight_decay=0.0001)

cfg.lr_config.warmup = None
cfg.log_config.interval = 10

cfg.lr_config.policy = 'step'

cfg.evaluation.metric = 'mAP'

cfg.evaluation.interval = 1

cfg.checkpoint_config.interval = 12

cfg.seed = 0

cfg.runner.max_epochs = 100

set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

cfg.device='cuda'

print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='FasterRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_w

In [8]:
!mkdir /content/drive/MyDrive/warship_detect_project/tutorial_exps_normal/blur_datasets

In [6]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

datasets = [build_dataset(cfg.data.train)]

self.data_root /content/drive/MyDrive/warship_detect_project/blur_datasets/ self.ann_file: /content/drive/MyDrive/warship_detect_project/blur_datasets/train.txt self.img_prefix:


/usr/local/lib/python3.10/dist-packages/mmdet-2.28.2-py3.10.egg/mmdet/datasets/custom.py:181: UserWarning: CustomDataset does not support filtering empty gt images.
  warnings.warn(


In [7]:
datasets

[
 WarshipDataset Train dataset with number of images 1697, and instance counts: 
 +---------------+-------+----------+-------+----------+-------+----------+-------+----------+-------+
 | category      | count | category | count | category | count | category | count | category | count |
 +---------------+-------+----------+-------+----------+-------+----------+-------+----------+-------+
 | 0 [DDG]       | 152   | 1 [DDH]  | 208   | 2 [FFG]  | 164   | 3 [FF]   | 194   | 4 [PCC]  | 222   |
 | 5 [PKG]       | 210   | 6 [LST]  | 203   | 7 [MLS]  | 136   | 8 [AOE]  | 186   | 9 [ATS]  | 110   |
 |               |       |          |       |          |       |          |       |          |       |
 | 10 [HULL_NUM] | 1787  |          |       |          |       |          |       |          |       |
 +---------------+-------+----------+-------+----------+-------+----------+-------+----------+-------+]

In [8]:
model = build_detector(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
model.CLASSES = datasets[0].CLASSES

In [9]:
%cd mmdetection

mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

/content/mmdetection


2024-05-13 10:45:34,212 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.


self.data_root /content/drive/MyDrive/warship_detect_project/blur_datasets/ self.ann_file: /content/drive/MyDrive/warship_detect_project/blur_datasets/val.txt self.img_prefix:


2024-05-13 10:45:37,121 - mmdet - INFO - load checkpoint from local path: /content/drive/MyDrive/warship_detect_project/tutorial_exps_normal/latest.pth
2024-05-13 10:45:37,742 - mmdet - INFO - Start running, host: root@b6736f8059b5, work_dir: /content/drive/MyDrive/warship_detect_project/tutorial_exps_normal/blur_datasets
2024-05-13 10:45:37,744 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 190/190, 10.3 task/s, elapsed: 18s, ETA:     0s
---------------iou_thr: 0.5---------------


...

2024-05-13 20:49:49,214 - mmdet - INFO - 
+----------+-----+------+--------+-------+
| class    | gts | dets | recall | ap    |
+----------+-----+------+--------+-------+
| DDG      | 14  | 59   | 1.000  | 0.887 |
| DDH      | 32  | 92   | 1.000  | 0.906 |
| FFG      | 20  | 102  | 1.000  | 0.707 |
| FF       | 26  | 64   | 1.000  | 0.967 |
| PCC      | 22  | 64   | 1.000  | 0.964 |
| PKG      | 28  | 41   | 0.929  | 0.923 |
| LST      | 20  | 85   | 1.000  | 0.979 |
| MLS      | 8   | 72   | 1.000  | 0.779 |
| AOE      | 22  | 61   | 1.000  | 0.990 |
| ATS      | 4   | 46   | 1.000  | 1.000 |
| HULL_NUM | 196 | 247  | 0.969  | 0.961 |
+----------+-----+------+--------+-------+
| mAP      |     |      |        | 0.915 |
+----------+-----+------+--------+-------+
2024-05-13 20:49:49,551 - mmdet - INFO - Epoch(val) [98][190]	AP50: 0.9150, mAP: 0.9147
2024-05-13 20:49:55,903 - mmdet - INFO - Epoch [99][10/849]	lr: 1.250e-06, eta: 0:11:26, time: 0.621, data_time: 0.236, memory: 2466, loss_

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 190/190, 9.3 task/s, elapsed: 20s, ETA:     0s
---------------iou_thr: 0.5---------------


2024-05-13 20:56:00,753 - mmdet - INFO - 
+----------+-----+------+--------+-------+
| class    | gts | dets | recall | ap    |
+----------+-----+------+--------+-------+
| DDG      | 14  | 59   | 1.000  | 0.887 |
| DDH      | 32  | 91   | 1.000  | 0.906 |
| FFG      | 20  | 102  | 1.000  | 0.707 |
| FF       | 26  | 64   | 1.000  | 0.967 |
| PCC      | 22  | 65   | 1.000  | 0.964 |
| PKG      | 28  | 41   | 0.929  | 0.927 |
| LST      | 20  | 85   | 1.000  | 0.979 |
| MLS      | 8   | 72   | 1.000  | 0.779 |
| AOE      | 22  | 60   | 1.000  | 0.990 |
| ATS      | 4   | 46   | 1.000  | 1.000 |
| HULL_NUM | 196 | 247  | 0.969  | 0.961 |
+----------+-----+------+--------+-------+
| mAP      |     |      |        | 0.915 |
+----------+-----+------+--------+-------+
2024-05-13 20:56:00,930 - mmdet - INFO - Epoch(val) [99][190]	AP50: 0.9150, mAP: 0.9151
2024-05-13 20:56:07,277 - mmdet - INFO - Epoch [100][10/849]	lr: 1.250e-06, eta: 0:05:41, time: 0.622, data_time: 0.237, memory: 2466, loss

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 190/190, 8.8 task/s, elapsed: 22s, ETA:     0s
---------------iou_thr: 0.5---------------


2024-05-13 21:02:15,778 - mmdet - INFO - 
+----------+-----+------+--------+-------+
| class    | gts | dets | recall | ap    |
+----------+-----+------+--------+-------+
| DDG      | 14  | 59   | 1.000  | 0.887 |
| DDH      | 32  | 91   | 1.000  | 0.906 |
| FFG      | 20  | 102  | 1.000  | 0.704 |
| FF       | 26  | 64   | 1.000  | 0.967 |
| PCC      | 22  | 65   | 1.000  | 0.964 |
| PKG      | 28  | 41   | 0.929  | 0.927 |
| LST      | 20  | 85   | 1.000  | 0.979 |
| MLS      | 8   | 72   | 1.000  | 0.779 |
| AOE      | 22  | 61   | 1.000  | 0.990 |
| ATS      | 4   | 46   | 1.000  | 1.000 |
| HULL_NUM | 196 | 247  | 0.969  | 0.961 |
+----------+-----+------+--------+-------+
| mAP      |     |      |        | 0.915 |
+----------+-----+------+--------+-------+
2024-05-13 21:02:15,959 - mmdet - INFO - Epoch(val) [100][190]	AP50: 0.9150, mAP: 0.9149
